# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [5]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [58]:
wine_dataset = load_wine()

x = wine_dataset['data'][:, 8:9]+ wine_dataset['data'][:, 10:13]
print(x.shape)
y = wine_dataset['target']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, random_state=17)




(178, 3)


**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [32]:
from sklearn.model_selection import StratifiedKFold, cross_val_score


In [46]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', multi_class='ovr', random_state=17)

print(cross_val_score(lr, x, y, cv=5))


[0.80555556 0.94444444 0.83333333 0.91428571 1.        ]


### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [55]:
lr = LogisticRegression(penalty='l2', multi_class='ovr', random_state=17)

skf = StratifiedKFold(n_splits=5, random_state=17, shuffle = True)
skf.get_n_splits(x, y)

print(skf)
accur_score = []

for train_index, test_index in skf.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr_model = lr.fit(X_train, y_train)
    y_predict = lr_model.predict(X_test)
    accur_score.append(accuracy_score(y_test, y_predict))
    print(cross_val_score(lr, X_test, y_test, cv=5))
print(np.mean(accur_score))



StratifiedKFold(n_splits=5, random_state=17, shuffle=True)
[0.875      0.85714286 0.57142857 1.         0.85714286]
[0.75       1.         1.         0.71428571 0.85714286]
[0.75       0.71428571 1.         1.         0.85714286]
[0.85714286 1.         1.         0.57142857 1.        ]
[0.85714286 1.         1.         1.         0.85714286]
0.8992063492063492


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [56]:
from sklearn.svm import SVC

In [57]:
svc_m = SVC(random_state=17)


for train_index, test_index in skf.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc_model = svc_m.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)
    accur_score.append(accuracy_score(y_test, y_predict))
    print(cross_val_score(svc_model, X_test, y_test, cv=5))
print(np.mean(accur_score))




[0.75       0.57142857 0.42857143 0.71428571 0.71428571]
[0.75       0.71428571 0.71428571 0.57142857 0.71428571]
[0.625      0.42857143 0.57142857 0.85714286 0.57142857]
[0.71428571 0.71428571 0.71428571 0.71428571 0.71428571]
[0.71428571 0.71428571 0.71428571 0.71428571 0.71428571]
0.787063492063492


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

In [65]:
y_predict1 = lr_model.predict(x)
con_mat = confusion_matrix(y, y_predict)
print('Матрица ошибок для LogisticRegression:')
print(con_mat)
y_predict2 = svc_model.predict(x)
con_mat = confusion_matrix(y, y_predict)
print('Матрица ошибок для SVC:')
print(con_mat)
#Как видим LogisticRegression лишь немного путает класс_1 с класс_0 и класс_2. Нулевой же и второй классы 
#распознаются достаточно точно
#SVC же метод имеет значительно хужудшие результаты и относит много объектов первого класа к второму и наоборот.


Матрица ошибок для LogisticRegression:
[[53  0  6]
 [ 4 56 11]
 [ 5 29 14]]
Матрица ошибок для SVC:
[[53  0  6]
 [ 4 56 11]
 [ 5 29 14]]


### 5.
Для каждой модели выведите classification report.

In [67]:
print('Рапорт LogisticRegression: ')
print(classification_report(y, y_predict1)) 
print('Рапорт SVC: ')
print(classification_report(y, y_predict2))






Рапорт LogisticRegression: 
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        59
           1       0.91      0.86      0.88        71
           2       0.86      0.92      0.89        48

    accuracy                           0.91       178
   macro avg       0.91      0.91      0.91       178
weighted avg       0.91      0.91      0.91       178

Рапорт SVC: 
              precision    recall  f1-score   support

           0       0.85      0.90      0.88        59
           1       0.66      0.79      0.72        71
           2       0.45      0.29      0.35        48

    accuracy                           0.69       178
   macro avg       0.66      0.66      0.65       178
weighted avg       0.67      0.69      0.67       178

